In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import ROOT as root

Welcome to JupyROOT 6.24/02


In [3]:
I_POS = np.array([i for i in np.arange(0.1, 1.4, 0.1)])

In [4]:
I_POS


array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3])

In [5]:
I_NEG = I_POS[::-1]

In [6]:
I_NEG


array([1.3, 1.2, 1.1, 1. , 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1])

In [7]:
import uproot
import uproot3

In [8]:
_data0 = uproot.open('../dati/dataRUN_20220522_18.10.41.977024_HALL.root')

In [9]:
_data0.keys()

['data_BPOS;2', 'data_BPOS;1', 'data_BNEG;2', 'data_BNEG;1']

In [10]:
BPOS = _data0['data_BPOS']
BNEG = _data0['data_BNEG']

In [11]:
BPOS.keys()

['ID', 'RUN', 'BUFSIZE', 'V_gen_lo', 'V_gen_hi', 'V_out_lo', 'V_out_hi']

In [14]:
V_out_std = np.array([BNEG['V_out_hi'].array(library='np'), 
                      BPOS['V_out_hi'].array(library='np')]).std()
V_out_std

0.0025601029582906316

In [19]:
V_out = np.array([BNEG['V_out_hi'].array(library='np'), 
                  BPOS['V_out_hi'].array(library='np')]).mean()

V_out


4.998965462048848

In [15]:
import setup_hall.linear_regression as lg

i10mA = lg.linear_regression('generatore.txt', system='current')

Applying ATLAS style settings


In [17]:
i10mA.set_model_parameters(1000/500, 0)
p0,p1 = i10mA.fit_model()

model results chi2/ndf (prob): 0.27968815631621563/9 (0.9999975624782381)
 FCN=0.279688 FROM MIGRAD    STATUS=CONVERGED      50 CALLS          51 TOTAL
                     EDM=7.73267e-13    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.72418e+00   3.00381e-02   8.48703e-06  -6.66866e-05
   2  p1           6.73130e-02   8.89186e-02   2.51236e-05  -1.14329e-05


In [18]:
p0

(1.7241819129116982, 0.030038068116462972)

In [21]:
V_out*p0[0]/1000

0.008619125832934895

In [23]:
from uncertainties import ufloat

V_out_u = ufloat(V_out, V_out_std)
gain_gen = ufloat(p0[0], p0[1])/1000
current = V_out*gain_gen

In [24]:
current.std_dev/current.nominal_value

0.017421635090543564

In [30]:
(current*1000).format('.3uS')

'8.619(150)'